In [1]:
import os
import yaml
import pytest
import pandas as pd

from network_wrangler import ProjectCard

from update_registry import update_registry
from read_project_cards import read_project_cards
from update_registry_database import update_registry_database

## defaults

In [2]:
CARD_DIR = os.path.join(".", "reference_projects")
REGISTRY_FILE = "registry.csv"
CONFIG_FILE = "registry_config.yml"

In [3]:
input_reg_df = pd.read_csv(REGISTRY_FILE)
input_reg_df

,type,id,project_added


In [4]:
with open(CONFIG_FILE, "r") as file:
        config_dict = yaml.safe_load(file)
config_dict

{'start_node_number': 1000, 'start_link_number': 500}

## test methods

In [5]:
card_file_list = read_project_cards(CARD_DIR)
card_file_list

[(<network_wrangler.projectcard.ProjectCard at 0x7f85f1e53210>,
  './reference_projects/project_B.yml'),
 (<network_wrangler.projectcard.ProjectCard at 0x7f85f256e9d0>,
  './reference_projects/project_A.yml')]

In [6]:
df = update_registry_database(card_file_list, input_reg_df, config_dict, False)
df

,type,id,project_added
0,node,1001,Project B
1,node,1002,Project B
2,link,501,Project B
3,node,1003,Project A
4,node,1004,Project A
5,link,502,Project A


In [7]:
update_registry(
        CONFIG_FILE, REGISTRY_FILE, "delete_me.csv", CARD_DIR, False,
    )

## tests

In [8]:
output_file = "test_update_registry.csv"

update_registry(
        config_file="registry_config.yml",
        input_reg_file="registry.csv",
        output_reg_file=output_file,
        card_dir=os.path.join(".", "reference_projects"),
        write_card_updates=False,
)

data = [
        ["node", 1001, "Project B"],
        ["node", 1002, "Project B"],
        ["node", 1003, "Project A"],
        ["node", 1004, "Project A"],
        ["link", 501, "Project B"],
        ["link", 502, "Project A"],
]
    
target_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_df = target_df.sort_values(by=["type", "id"]).reset_index(drop=True)

outcome_df = pd.read_csv(output_file)
outcome_df = outcome_df.sort_values(by=["type", "id"]).reset_index(drop=True)

In [9]:
target_df

,type,id,project_added
0,link,501,Project B
1,link,502,Project A
2,node,1001,Project B
3,node,1002,Project B
4,node,1003,Project A
5,node,1004,Project A


In [10]:
outcome_df

,type,id,project_added
0,link,501,Project B
1,link,502,Project A
2,node,1001,Project B
3,node,1002,Project B
4,node,1003,Project A
5,node,1004,Project A


In [11]:
target_df.equals(outcome_df)

True

In [16]:
output_file = "test_update_registry.csv"
temp_file = "temp_registry.csv"
data = [
        ["node", 1001, "Project Z"],
        ["node", 1002, "Project Z"],
        ["link", 501, "Project Z"],
    ]
temp_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
temp_df.to_csv(temp_file, index=False)

update_registry(
        config_file="registry_config.yml",
        input_reg_file=temp_file,
        output_reg_file=output_file,
        card_dir=os.path.join(".", "reference_projects"),
        write_card_updates=False,
    )

data = [
        ["node", 1001, "Project Z"],
        ["node", 1002, "Project Z"],
        ["link", 501, "Project Z"],
        ["node", 1003, "Project B"],
        ["node", 1004, "Project B"],
        ["node", 1005, "Project A"],
        ["node", 1006, "Project A"],
        ["link", 502, "Project B"],
        ["link", 503, "Project A"],
    ]
target_df = pd.DataFrame(data, columns=["type", "id", "project_added"])
target_df = target_df.sort_values(by=["type", "id"]).reset_index(drop=True)

outcome_df = pd.read_csv(output_file)
outcome_df = outcome_df.sort_values(by=["type", "id"]).reset_index(drop=True)

In [17]:
target_df

,type,id,project_added
0,link,501,Project Z
1,link,502,Project B
2,link,503,Project A
3,node,1001,Project Z
4,node,1002,Project Z
5,node,1003,Project B
6,node,1004,Project B
7,node,1005,Project A
8,node,1006,Project A


In [18]:
outcome_df

,type,id,project_added
0,link,501,Project Z
1,link,502,Project B
2,link,503,Project A
3,node,1001,Project Z
4,node,1002,Project Z
5,node,1003,Project B
6,node,1004,Project B
7,node,1005,Project A
8,node,1006,Project A


In [19]:
target_df.equals(outcome_df)

True